In [1]:
strings=['20, t:catcher in the rye, i:axl rose, b:madagascar, c:2008 albums','30, b:Taxicab 1729',
'50, b:Marc Spector, i:Marvel Comics, c:1980 comics debuts']

In [21]:
splits=strings[2].split(',')

In [22]:
splits[0]

'50'

In [24]:
' '.join(splits[1:]).strip()

'b:Marc Spector  i:Marvel Comics  c:1980 comics debuts'

In [19]:
' '.join(splits[1:])

' t:catcher in the rye  i:axl rose  b:madagascar  c:2008 albums'

# Indexing

In [ ]:
import re
from collections import defaultdict
import xml.etree.ElementTree as etree
import nltk
nltk_stemmer=nltk.stem.SnowballStemmer('english')
from tqdm import tqdm
import sys
import os
from time import time

# INDEX_FILE='inverted_index.txt'
# WIKI_XML_FILE_NAME='enwiki-latest-pages-articles2.xml'

# sample_xml='sample.xml'
# title_file='title_index.txt'
class Wiki_Indexer:
    def __init__(self):
        self.title_index=defaultdict(list)
        self.text_index=defaultdict(list)
        self.category_index=defaultdict(list)
        #title_tag_file=open(title_file,'w',encoding='utf-8')
        
        self.title_positions=[]
        self.stopwords=defaultdict()
        self.infobox_index=defaultdict(list)
        self.reference_index=defaultdict(list)
        self.stemmed_words={}
        self.css_pattern=re.compile(r'{\|(.*?)\|}',re.DOTALL)
        self.file_pattern = re.compile(r'\[\[file:(.*?)\]\]',re.DOTALL)
        self.url_pattern=re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',re.DOTALL)
        
        self.STOPWORDS_FILE='stopwords2.txt'

        self.title_index_file=open('title_names.txt','w',encoding='utf-8')

        self.total_tokens_in_dump=0
        self.total_tokens_in_index=0
        
        self.page_count=0
        
        self.dump_size=0

   
        self.num_file=0

        self.num_files=12
    
    def load_stopwords(self):
            stopwords_file=open(self.STOPWORDS_FILE,'r')
            stpwords=[line.rstrip() for line in stopwords_file]
            for word in stpwords:
                self.stopwords[word]=True
            stopwords_file.close()
    
    def add_to_stem_dict(self,word):
       
        self.stemmed_words[word]=nltk_stemmer.stem(word)
        return self.stemmed_words[word]
        
    def parse_category_box(self,text,category_count):
            categories=re.findall("\[\[Category:(.*?)\]\]",text)
      
            for category in categories:
                text_pattern=re.compile(r'[a-zA-Z]+|[0-9]{1,4}') 
                category_words=re.findall(text_pattern,category)
                category_words=list(map(lambda x:x.lower(),category_words))
                
                category_words=[x for x in category_words if (x!="" and (not x in self.stopwords and len(x)) and len(x)>2)]
                #self.total_tokens_in_dump+=len(category_words)
                category_words=[self.stemmed_words[word] if word in self.stemmed_words else self.add_to_stem_dict(word) for word in category_words]
                # print(category_words)
                for word in category_words:
                     category_count[word]+=1
            
            return category_count
    

        
    def parse_infobox_text(self,text,infobox_page_count):
     
                infobox_text=[]
                flag=False
                for line in text.split('\n'):
                    if line.startswith("{{Infobox"):
                        flag=True
                        infobox_text.append(line)
                    elif flag and line!="}}":
                        infobox_text.append(line)
                    elif flag:
                        break
                
                for info_text in infobox_text:
                    text_pattern=re.compile(r'[a-zA-Z]+|[0-9]{1,4}') 
                    info_words=re.findall(text_pattern,info_text)
                    
                    info_words=list(map(lambda x:x.lower(),info_words))
                    
                    info_words=[x for x in info_words if (x!="" and (not x in self.stopwords)and len(x)>2)]
                    #self.total_tokens_in_dump+=len(info_words)

                    info_words=[self.stemmed_words[word] if word in self.stemmed_words else self.add_to_stem_dict(word) for word in info_words]
                    for word in info_words:
                        infobox_page_count[word]+=1
                return infobox_page_count
    
                
    def parse_main_text(self,text,main_text_count,category_count,infobox_page_count,reference_count):
            #text=css_pattern.sub('',text)
            body=text.lower()
           
            text_pattern=re.compile(r'[a-zA-Z]+|[0-9]{1,4}') 
            body=re.findall(text_pattern,body)
            body_words=[x for x in body if (x!="" and (not x in self.stopwords)and len(x)>2)]
            
            self.total_tokens_in_dump+=len(body_words)

            body_words=[self.stemmed_words[word] if word in self.stemmed_words else self.add_to_stem_dict(word) for word in body_words]
            for word in body_words:
                main_text_count[word]+=1
            
            for word in main_text_count.keys():
                 
                if(category_count.get(word) is not None):
                    main_text_count[word]-=category_count.get(word)

                if(infobox_page_count.get(word) is not None):
                    main_text_count[word]-=infobox_page_count.get(word)


                if(reference_count.get(word)is not None):
                    main_text_count[word]-=reference_count.get(word)


            return main_text_count
                
    def parse_title(self,title,title_count):

            ttle=title
            ttle+='\n'
            
            body=title.lower()
            text_pattern=re.compile(r'[a-zA-Z]+|[0-9]{1,4}') 
            body=re.findall(text_pattern,body)
            body_words=[x for x in body if (x!="" and len(x)>2 and (not x in self.stopwords))]
            
            self.total_tokens_in_dump+=len(body_words)
            body_words=[self.stemmed_words[word] if word in self.stemmed_words else self.add_to_stem_dict(word) for word in body_words]
            
            for word in body_words:
                title_count[word]+=1
            return title_count

    def parse_for_references(self,text,reference_count):
  

        text_pattern=re.compile("[a-zA-Z\d]+")
        st1=re.sub('&lt;ref&gt;','<ref>',text)
        st2=re.sub('&lt;/ref&gt;','</ref>',st1)
        lists=re.findall('\<ref\>([^\<]+)\<\/ref\>',st2)
        
        words=[re.findall(text_pattern,element) for element in lists]
        nonempty_words=[[word for word in list1 if(word!="" and len(word)>2 and (not word in self.stopwords))] for list1 in words]
        words=[word.lower() for lists in nonempty_words for word in lists]
        #self.total_tokens_in_dump+=len(words)

        words=[self.stemmed_words[word] if word in self.stemmed_words else self.add_to_stem_dict(word) for word in words]
        
        for word in words:
            reference_count[word]+=1
        return reference_count
    
    def write_title_index(self,id_,count,title_count):
         
            for word in title_count:
                word_count=str(title_count[word])
                page_number=str(count)
                word_and_page_count=id_+':'+word_count
                self.title_index[word].append(word_and_page_count)
        
    def write_text_index(self,id_,count,main_text_count):
           for word in main_text_count:
                if main_text_count[word]>0:
                    word_count=str(main_text_count[word])
                    page_number=str(count)
                    word_and_page_count=id_+':'+word_count
                    self.text_index[word].append(word_and_page_count)
        
        
    def write_category_index(self,id_,count,category_count):
            for word in category_count:
                if category_count[word]>0:
                    word_count=str(category_count[word])
                    page_number=str(count)
                    word_and_page_count=id_+':'+word_count
                    self.category_index[word].append(word_and_page_count)
        

    def write_reference_index(self,id_,count,reference_count):
          
            for word in reference_count:
            	if reference_count[word]>0:
	                word_count=str(reference_count[word])
	                page_number=str(count)
	                word_and_page_count=id_+':'+word_count
	                self.reference_index[word].append(word_and_page_count)
       

                
    def write_infobox_index(self,id_,count,infobox_page_count):
            for word in infobox_page_count:
                if infobox_page_count[word]!=0:
                    word_count=str(infobox_page_count[word])
                    page_number=str(count)
                    word_page_count=id_+':'+word_count

                    self.infobox_index[word].append(word_page_count)
                else:
                    continue


    def get_params(self):
            param=sys.argv
            self.collectionFile='xml_dump.xml'#param[1]
            self.indexFile='index_file.txt'#param[2]
            self.stat_file='index_stat.txt'#param[3]
            
    def parse_xml_file(self):
            from time import time

            
            #self.get_params()
            self.load_stopwords()
            count=0
            revsn_tag=False
            #WIKI_XML_FILE_NAME
            
            self.num_file=1
            dump_names=os.listdir(sys.argv[1])
            file_name=sys.argv[1]+'/'
            print(dump_names)
            for i in range(len(dump_names)):
                #self.collectionFile=file_name+'_'+str(i)+'.xml'
                self.collectionFile=dump_names[i]
          
                print(f'Processing dump number: {i}')
                start=time()
                with open(file_name+self.collectionFile,encoding='utf-8') as xmlfile:
                    context=etree.iterparse(xmlfile,events=('start','end'))
                    context=iter(context)
                    
                    for event, element in tqdm(context):
                        tag_name=re.sub(r'{.*}','',element.tag)
                        if event=='start':
                            if tag_name=='page':
                                category_count=defaultdict(int)
                                title_count=defaultdict(int)
                                main_text_count=defaultdict(int)
                                infobox_page_count=defaultdict(int)
                                reference_count=defaultdict(int)
                                revsn_tag=False
                                self.page_count+=1
                            elif tag_name=='revision':
                                revsn_tag=True

                        else:
                            if tag_name=='title' :
                                title_text=element.text
                                title_count=self.parse_title(title_text,title_count)

                            elif tag_name=='id' and not revsn_tag:
                                ids=int(element.text)

                            elif tag_name=='text':
                                try:
                                    current_text=element.text
                                    current_text=self.css_pattern.sub('',current_text)
                                    current_text=self.file_pattern.sub('',current_text)
                                    current_text=self.url_pattern.sub('',current_text)

                                    category_count=self.parse_category_box(current_text,category_count)
                            
                                    infobox_page_count=self.parse_infobox_text(current_text,infobox_page_count)
                                    reference_count=self.parse_for_references(current_text,reference_count)
                                    main_text_count=self.parse_main_text(current_text,main_text_count,category_count,infobox_page_count,reference_count)
                            
                                except:
                                    pass

                            elif tag_name=='page':
                                self.write_title_index(str(ids),count,title_count)
                                self.write_category_index(str(ids),count,category_count)
                                self.write_text_index(str(ids),count,main_text_count)
                                self.write_infobox_index(str(ids),count,infobox_page_count)
                                self.write_reference_index(str(ids),count,reference_count)
                                self.title_index_file.write(str(ids)+'#'+title_text+'\n')
                         

                                if(self.page_count%30000==0):
                                    self.write_index_files(self.num_file)
                                    self.num_file+=1
                                    self.text_index.clear()
                                    self.title_index.clear()
                                    self.infobox_index.clear()
                                    self.category_index.clear()
                                    self.reference_index.clear()
                                
                                if(self.page_count%50000==0):
                                    self.stemmed_words.clear()

                            element.clear()

                   

                    end=time()
                    print(f'Parsing successful!! in {end-start} seconds ')
                    
                    self.write_index_files(self.num_file)
                    self.num_file+=1

                    self.text_index.clear()
                    self.title_index.clear()
                    self.infobox_index.clear()
                    self.category_index.clear()
                    self.reference_index.clear()

                    xmlfile.close()
            self.title_index_file.close()
                    
                    #title_tag_file.close()


    def write_index_files(self,idx):

            category_set=sorted(set(self.category_index.keys()))
            title_set=sorted(set(self.title_index.keys()))
            text_set=sorted(set(self.text_index.keys()))
            reference_set=sorted(set(self.reference_index.keys()))
            infobox_set=sorted(set(self.infobox_index.keys()))


            for word in self.stopwords.keys():
                    try:
                        category_set.remove(word)
                        title_set.remove(word)
                        text_set.remove(word)
                        reference_set.remove(word)
                        infobox_set.remove(word)
                    except:
                        pass
                    
                    
                    
            body_directory='indexes/body/'
            category_directory='indexes/category/'
            references_directory='indexes/references/'
            infobox_directory='indexes/infobox/'
            title_directory='indexes/titles/'

            category_index_file=category_directory+'category_index_'+str(idx)+'.txt'
            infobox_index_file=infobox_directory+'infobox_index_'+str(idx)+'.txt'
            reference_index_file=references_directory+'reference_index_'+str(idx)+'.txt'
            body_index_file=body_directory+'body_index_'+str(idx)+'.txt'
            title_index_file=title_directory+'title_index_'+str(idx)+'.txt'

            cif=open(category_index_file,'w',encoding='utf-8')
            iif=open(infobox_index_file,'w',encoding='utf-8')
            rif=open(reference_index_file,'w',encoding='utf-8')
            bif=open(body_index_file,'w',encoding='utf-8')
            tif=open(title_index_file,'w',encoding='utf-8')

            self.total_tokens_in_index=self.total_tokens_in_index+len(category_set)+len(title_set)+len(text_set)+len(reference_set)+len(infobox_set)
              
            for i in category_set:
                            cat_posting=self.category_index[i]
                            cat_posting=','.join(cat_posting)
                            if cat_posting:
                                cif.write(f'{i}|{cat_posting}\n')

            for i in infobox_set:
                            info_posting=self.infobox_index[i]
                            info_posting=','.join(info_posting)
                            if info_posting:
                                iif.write(f'{i}|{info_posting}\n')
            
            for i in reference_set:
                            reference_posting=self.reference_index[i]
                            reference_posting=','.join(reference_posting)
                            if reference_posting:
                                rif.write(f'{i}|{reference_posting}\n')
            
            for i in  text_set:
                            text_posting=self.text_index[i]
                            text_posting=','.join(text_posting)
                            if text_posting:
                                bif.write(f'{i}|{text_posting}\n')                    

            for i in title_set:
                            title_posting=self.title_index[i]
                            title_posting=','.join(title_posting)
                            if title_posting:
                                tif.write(f'{i}|{title_posting}\n')


            cif.close()
            bif.close()
            tif.close()
            iif.close()
            rif.close()




            # stat_file=open(f'index_stat_{idx}.txt','w')
            # stat_file.write(str(self.total_tokens_in_dump)+'\n')
            # stat_file.write(str(self.total_tokens_in_index)+'\n')
            # stat_file.close()



def get_size(start_path = 'indexes'):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(start_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            # skip if it is symbolic link
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)

    return total_size


if __name__=='__main__':
    indexer=Wiki_Indexer()

    os.makedirs('indexes/body')
    os.makedirs('indexes/titles')
    os.makedirs('indexes/references')
    os.makedirs('indexes/category')
    os.makedirs('indexes/infobox')
    

    start=time()
    indexer.parse_xml_file()
    end=time()
    

    index_size=get_size()/2**30

   
    stat_file=open('stats.txt','w',encoding='utf-8')
    stat_file.write(f'{index_size} GB\n')
    stat_file.write(f'{indexer.num_file-1}\n')
    stat_file.write(f'{indexer.total_tokens_in_index}\n')
   
    stat_file.close()

    


    print(f'Total time taken is {end-start} Seconds')

In [ ]:

# if __name__=='__main__':
#     indexer=Wiki_Indexer()

#     os.makedirs('indexes/body')
#     os.makedirs('indexes/titles')
#     os.makedirs('indexes/references')
#     os.makedirs('indexes/category')
#     os.makedirs('indexes/infobox')
    

#     start=time()
#     indexer.parse_xml_file()
#     end=time()
    

#     index_size=get_size()/2**30

   
#     stat_file=open('stats.txt','w',encoding='utf-8')
#     stat_file.write(f'{index_size} GB\n')
#     stat_file.write(f'{indexer.num_file-1}\n')
#     stat_file.write(f'{indexer.total_tokens_in_index}\n')
   
#     stat_file.close()

    


#     print(f'Total time taken is {end-start} Seconds')

In [ ]:
import os
from heapq import *
from collections import defaultdict
import math
from tqdm import tqdm
import operator
from time import time
import sys


class IndexMerger:
    def __init__(self,top_k,doc_count):
        
        self.body_directory='indexes/body/'
        self.category_directory='indexes/category/'
        self.references_directory='indexes/references/'
        self.infobox_directory='indexes/infobox/'
        self.title_directory='indexes/titles/'


        self.body_files=os.listdir(self.body_directory)
        self.body_index='final_text_index.txt'
        

        self.title_files=os.listdir(self.title_directory)
        self.title_index='final_title_index.txt'

        self.category_files=os.listdir(self.category_directory)
        self.category_index='final_category_index.txt'

        self.reference_files=os.listdir(self.references_directory)
        self.reference_index='final_reference_index.txt'


        self.infobox_files=os.listdir(self.infobox_directory)
        self.infobox_index='final_infobox_index.txt'

        self.top_k=top_k
        self.num_pages=doc_count


    def get_tf_idf(self,word,postings,index_writer,k):
         
        word_idf=defaultdict(float)

        docs=postings.split(',')
       
        idf=math.log10(self.num_pages/len(docs))
        for doc in docs:
                doc_num,count=doc.split(':')
                tf=1+math.log10(int(count))
                word_idf[str(doc_num)]=round(idf*tf,2)

        
        word_idf=sorted(word_idf.items(),key=operator.itemgetter(1),reverse=True)
 
        final_doc=[str(item[0])+':'+str(item[1]) for item in word_idf]


        final_doc=final_doc[:k+1]
        
        tf_idf_index=','.join(final_doc)+'\n'
        
        tf_idf_index=word+'|'+tf_idf_index
        
        index_writer.write(tf_idf_index)

 
    def merge_index_files(self,heap,file_pointers,index_type,index_files,k):
        
        final_index=open(index_type,'w',encoding='utf-8')
        num_files=len(index_files)
       
        line_number=0
        file_count=0
        heapify(heap)

        
        
        try:
            while(file_count<=num_files):

                posting_list,file_number=heappop(heap)
                word,postings=posting_list.split('|')
            
                fp=file_pointers[int(file_number)]
                next_line=fp.readline().rstrip()
                
            
                if next_line:
                    heappush(heap,(next_line,file_number))
                
                else:
                    fp.close()
                    file_count=file_count+1

                
                if file_count==num_files:
                    break

            
                while(1):
                    posts,file_number=heappop(heap)
                
                    curr_word,curr_postings=posts.split('|')

                    if curr_word==word:
                        postings=postings+","+curr_postings
                        fp=file_pointers[int(file_number)]
                        next_line=fp.readline().rstrip()
                        
                        if next_line:
                            heappush(heap,(next_line,file_number))

                        else:
                            fp.close()
                            file_count=file_count+1
                    else:
                        heappush(heap,(posts,file_number))
                        break;

                self.get_tf_idf(word,postings,final_index,k)
        except:
            pass
        final_index.close()


    
    def create_scored_index(self,index_file,scored_index_file_name,top_k):
        big_index=open(index_file,'r',encoding='utf-8')
        large_dict=defaultdict(str)

        for lines in big_index:
            word,postings=lines.split('|')
            if(large_dict.get(word) is not None):
                    postings=','+postings
            large_dict[word]+=postings.rstrip()
        
        big_index.close()

        new_dict=defaultdict(list)
        
        for word in tqdm(large_dict.keys()):
            posts=large_dict[word].split(',')
    
            postings=[(str(docs.split(':')[0]),float(docs.split(':')[1])) for docs in posts]
            top=sorted(postings,key=operator.itemgetter(1),reverse=True)
            
            # if(len(top)>top_k):
            #     new_dict[word]=top[:top_k+1]
            # else:
            #     new_dict[word]=top

            new_dict[word]=top[:top_k+1]

        scored_index=open(scored_index_file_name,'w',encoding='utf-8')

        for word in tqdm(sorted(new_dict.keys())):
            curr=""
            for items in new_dict[word]:
                curr+=items[0]+':'+str(items[1])+','
            
            scored_index.write(word+'|'+curr.rstrip(',')+'\n')

        scored_index.close()


    def merge_body_files(self):
        self.body_file_pointers={}
        
        i=1
        heap=[]

        for fname in self.body_files:
            fp=open(self.body_directory+fname,'r',encoding='utf-8')
            self.body_file_pointers[i]=fp
            first_line=fp.readline().rstrip()
            heap.append((first_line,i))
            i+=1

        start=time()
        self.merge_index_files(heap,self.body_file_pointers,self.body_index,self.body_files,self.top_k)
        end=time()

        print(f'Time taken to construct merged body files {end-start}')
        

        start2=time()
       
        self.create_scored_index(self.body_index,"scored_body_index.txt",self.top_k)

        end2=time()
        print(f'time taken to create scored body index {end2-start2}')
        heap=[]
        
        self.clean_up("body")

    
    def merge_category_files(self):
        self.category_file_pointers={}
        
        i=1
        heap=[]

        for fname in self.category_files:
            fp=open(self.category_directory+fname,'r',encoding='utf-8')
            self.category_file_pointers[i]=fp
            first_line=fp.readline().rstrip()
            heap.append((first_line,i))
            i+=1

        start=time()
        self.merge_index_files(heap,self.category_file_pointers,self.category_index,self.category_files,1000000)

        end=time()

        print(f'Time taken to construct merged category_files files {end-start}')
        

        start2=time()
       
        self.create_scored_index(self.category_index,"scored_category_index.txt",1000000)

        end2=time()
        print(f'time taken to create scored category index {end2-start2}')
        heap=[]
        
        self.clean_up("category")
    
    
    def merge_infobox_files(self):
        self.infobox_file_pointers={}
        
        i=1
        heap=[]

        for fname in self.infobox_files:
            fp=open(self.infobox_directory+fname,'r',encoding='utf-8')
            self.infobox_file_pointers[i]=fp
            first_line=fp.readline().rstrip()
            heap.append((first_line,i))
            i+=1

        start=time()
        self.merge_index_files(heap,self.infobox_file_pointers,self.infobox_index,self.infobox_files,1000000)

        end=time()

        print(f'Time taken to construct merged infobox files {end-start}')
        

        start2=time()
       
        self.create_scored_index(self.infobox_index,"scored_infobox_index.txt",1000000)

        end2=time()
        print(f'time taken to create scored infobox index {end2-start2}')
        heap=[]
        
        self.clean_up("infobox")

    
    def merge_title_files(self):
        self.title_file_pointers={}
        
        i=1
        heap=[]

        for fname in self.title_files:
            fp=open(self.title_directory+fname,'r',encoding='utf-8')
            self.title_file_pointers[i]=fp
            first_line=fp.readline().rstrip()
            heap.append((first_line,i))
            i+=1

        start=time()
        self.merge_index_files(heap,self.title_file_pointers,self.title_index,self.title_files,1000000)

        end=time()

        print(f'Time taken to construct merged title files {end-start}')
        

        start2=time()
       
        self.create_scored_index(self.title_index,"scored_title_index.txt",1000000)

        end2=time()
        print(f'time taken to create scored title index {end2-start2}')
        heap=[]
        
        self.clean_up("title")



    def merge_reference_files(self):
        self.reference_file_pointers={}
        
        i=1
        heap=[]

        for fname in self.reference_files:
            fp=open(self.references_directory+fname,'r',encoding='utf-8')
            self.reference_file_pointers[i]=fp
            first_line=fp.readline().rstrip()
            heap.append((first_line,i))
            i+=1

        start=time()
        self.merge_index_files(heap,self.reference_file_pointers,self.reference_index,self.reference_files,2500)

        end=time()

        print(f'Time taken to construct merged reference files {end-start}')
        

        start2=time()
       
        self.create_scored_index(self.reference_index,"scored_reference_index.txt",2500)

        end2=time()
        print(f'time taken to create scored reference index {end2-start2}')
        heap=[]
        self.clean_up("reference")




    def clean_up(self,clean_up_type):
        if clean_up_type=="body":

            os.remove(self.body_index)

            for i in self.body_file_pointers:
                self.body_file_pointers[i].close()

            #for fname in self.body_files:
                #os.remove(self.body_directory+fname)

        if clean_up_type=="reference":

            os.remove(self.reference_index)

            for i in self.reference_file_pointers:
                self.reference_file_pointers[i].close()

            #for fname in self.reference_files:
                #os.remove(self.references_directory+fname)

        
        if clean_up_type=="category":

            os.remove(self.category_index)

            for i in self.category_file_pointers:
                self.category_file_pointers[i].close()

            #for fname in self.category_files:
                #os.remove(self.category_directory+fname)


        if clean_up_type=="infobox":

            os.remove(self.infobox_index)

            for i in self.infobox_file_pointers:
                self.infobox_file_pointers[i].close()

            #for fname in self.infobox_files:
                #os.remove(self.infobox_directory+fname)


        if clean_up_type=="title":

            os.remove(self.title_index)

            for i in self.title_file_pointers:
                self.title_file_pointers[i].close()

            #for fname in self.title_files:
                #os.remove(self.title_directory+fname)



    def merge_files(self):
        self.merge_title_files()
        self.merge_infobox_files()
        self.merge_category_files()
        self.merge_body_files()
        self.merge_reference_files()


if __name__=='__main__':
    start=time()
    doc_count=0
    tf=open('title_names.txt','r',encoding='utf-8')
    for line in tf:
        doc_count+=1
    tf.close()
    print(doc_count)
    
    merger=IndexMerger(520,doc_count)
    
    merger.merge_files()
    
    end=time()

    print(f'Total time taken to merge is {end-start} seconds!! ')

In [ ]:

# if __name__=='__main__':
    
#     doc_count=0
#     tf=open('title_names.txt','r',encoding='utf-8')
#     for line in tf:
#     	doc_count+=1
#     tf.close()
#     print(doc_count)
    
#     merger=IndexMerger(120,doc_count)

#     merger.merge_files()

In [ ]:
#     def write_index_files(self,idx):

#             final_set=set(sorted(set.union(set(self.category_index.keys()),set(self.title_index.keys()),set(self.reference_index.keys()),set(self.text_index.keys()),set(self.infobox_index.keys()))))
            
#             for word in self.stopwords.keys():
#                     try:
#                         final_set.remove(word)
#                     except:
#                         pass
      
#             # big_index='big_index.txt'
#             # big_index_file=open(self.indexFile,'w')
#             final_set=sorted(final_set)
#             self.total_tokens_in_index+=len(final_set)
#             index_file='index_num'+str(idx)+'.txt'
#             with open(index_file,'w',encoding='utf-8') as big_index_file:
#                     for i in final_set:
#                         cat_posting=self.category_index[i]
#                         cat_posting=','.join(cat_posting)
#                         if not cat_posting:
#                             cat_posting=""

#                         info_posting=self.infobox_index[i]
#                         info_posting=','.join(info_posting)
#                         if not info_posting:
#                             info_posting=""
                        
#                         text_posting=self.text_index[i]
#                         text_posting=','.join(text_posting)
#                         if not text_posting:
#                             text_posting=''

#                         title_posting=self.title_index[i]
#                         title_posting=','.join(title_posting)
#                         if not title_posting:
#                             title_posting=''
                    
#                         reference_posting=self.reference_index[i]
#                         reference_posting=','.join(reference_posting)
#                         if not reference_posting:
#                             reference_posting=''
                
#                         to_write=f'{i}|c:{cat_posting};i:{info_posting};b:{text_posting};t:{title_posting};r:{reference_posting}\n'
                        
#                         #tp.write(f'{i}:{str(big_index_file.tell())}\n')
                        
#                         big_index_file.write(to_write)                    
#             print('Index File written')
#             big_index_file.close()
           
#             stat_file=open(f'index_stat_{idx}.txt','w')
#             stat_file.write(str(self.total_tokens_in_dump)+'\n')
#             stat_file.write(str(self.total_tokens_in_index)+'\n')
#             stat_file.close()

# Search

In [ ]:
import nltk
from collections import defaultdict
import xml.etree.ElementTree as etree
nltk_stemmer=nltk.stem.SnowballStemmer('english')
from tqdm import tqdm
from collections import defaultdict
import sys
import operator
from time import time
import re



result_file=open('queries_op.txt','w',encoding='utf-8')

title_dict=defaultdict(str)
text_score_dict=defaultdict(str)
category_score_dict=defaultdict(str)
title_score_dict=defaultdict(str)
infobox_score_dict=defaultdict(str)
references_score_dict=defaultdict(str)
stopwords=defaultdict()


title_file=open('title_names.txt','r',encoding='utf-8')

body_scores_file=open('scored_body_index.txt','r',encoding='utf-8')
title_scores_file=open('scored_title_index.txt','r',encoding='utf-8')
infobox_scores_file=open('scored_infobox_index.txt','r',encoding='utf-8')
reference_scores_file=open('scored_reference_index.txt','r',encoding='utf-8')
category_scores_file=open('scored_category_index.txt','r',encoding='utf-8')



stopwords_file=open('stopwords2.txt','r')
stpwords=[line.rstrip() for line in stopwords_file]
for word in stpwords:
    stopwords[word]=True
stopwords_file.close()



for line in tqdm(title_file):
    try:
        ids,title=line.split('#')
        title_dict[int(ids)]=title.rstrip()
    except:
        pass


for line in tqdm(body_scores_file):
    word,scores=line.split('|')
    text_score_dict[word]=scores.rstrip()
    
for line in tqdm(reference_scores_file):
    word,scores=line.split('|')
    references_score_dict[word]=scores.rstrip()
    
for line in tqdm(category_scores_file):
    word,scores=line.split('|')
    category_score_dict[word]=scores.rstrip()
    
for line in tqdm(title_scores_file):
    word,scores=line.split('|')
    title_score_dict[word]=scores.rstrip()
    
for line in tqdm(infobox_scores_file):
    word,scores=line.split('|')
    infobox_score_dict[word]=scores.rstrip()
    
title_file.close()
body_scores_file.close()
reference_scores_file.close()
infobox_scores_file.close()
category_scores_file.close()
title_scores_file.close()


def is_phrase_query(query):
    ftq=['b:','c:','i:','r:','t:','l:']
    for q in ftq:
        if q in query:
            return False
    return True



def search_in_dictionary(word,dictionary_type):
    if dictionary_type=="body":
        return text_score_dict[word]
    
    if dictionary_type=="category":
        return category_score_dict[word]
    
    if dictionary_type=="infobox":
        return infobox_score_dict[word]
    
    if dictionary_type=="title":
        return title_score_dict[word]

    if dictionary_type=="references":
        return references_score_dict[word]



def get_one_word_scores(word,field_type):
    word=nltk_stemmer.stem(word)
    all_scores=search_in_dictionary(word,field_type)
    
    if all_scores != "":
        scores=all_scores.split(',')
        return scores
    else:
        return ""



def search_worst_case_query(query,num_results,sorted_ids,ordered_score_list):
    worst_case_id_score=defaultdict(float)
    print('got through worst case')
    new_ids=[ids for id_lst in sorted_ids for ids in id_lst ]
    
    for ids in new_ids:
        for dicts in ordered_score_list:
            worst_case_id_score[ids]+=dicts[ids]
    
    new_id_scores=sorted(worst_case_id_score.items(),key=operator.itemgetter(1),reverse=True)
    
    return new_id_scores




def get_relevant_titles(intersected_ids,query,sorted_ids,num_results,ordered_score_list):
    copy=query
    query=query.lower()
    #print('printing via rel fun '+query)
    words_in_query=set(query.split())
    
    relevant_ids=[]
    
    relevant_id_score=defaultdict(float)
    
    for id_list in sorted_ids:
        for rel_id in id_list:
            title_for_id=title_dict[rel_id].lower()
            words_in_title=set(title_for_id.split())
            #print(words_in_title)
            if len(words_in_title.intersection(words_in_query))>=len(words_in_query):
                relevant_ids.append(rel_id)
    

    for ids in relevant_ids:
        for dicts in ordered_score_list:
            relevant_id_score[ids]+=dicts[ids]
    

    relevant_id_score=sorted(relevant_id_score.items(),key=operator.itemgetter(1),reverse=True)

    return relevant_id_score




def search_in_index(query,query_type,num_results):
    query=query.lstrip()
    query=query.rstrip()
    query=query.lower()
    words=list(set(query.split()))
    
    for word in stopwords.keys():
        try:
            words.remove(word)
        except:
            pass
    
    
    ordered_score_list=[]
    sorted_score_list=[]

    for word in words:
   
        ordered_score=defaultdict(float)

        word_scores=get_one_word_scores(word,query_type)
       
        if word_scores!="":
            for item in word_scores:
                ordered_score[int(item.split(':')[0])]=float(item.split(':')[1])


            ordered_score_list.append(ordered_score)

            ordered_set=sorted(ordered_score.items(),key=operator.itemgetter(1),reverse=True)

            sorted_score_list.append(ordered_set)
        else:
            return ""


    sorted_ids=[]
    for sorted_scores in sorted_score_list:
        ids=[item[0] for item in sorted_scores]

        sorted_ids.append(ids)

    
    #print(sorted_ids)
    
    intersected_ids=set(sorted_ids[0]).intersection(*sorted_ids)
    
    if len(intersected_ids) < num_results:
        return get_relevant_titles(intersected_ids,query,sorted_ids,num_results,ordered_score_list)

    
    total_scores=defaultdict(float)
    for ids in intersected_ids:
        for dicts in ordered_score_list:
            total_scores[ids]+=dicts[ids]
    
    #print('printing  normally '+query)

    final_scores=sorted(total_scores.items(),key=operator.itemgetter(1),reverse=True)

    return final_scores




def search_phrase_query(query,num_results):
    
    titles=search_in_index(query,'title',num_results)
    bodys=search_in_index(query,'body',num_results)
    refs=search_in_index(query,'references',num_results)
    cats=search_in_index(query,'category',num_results)
    infos=search_in_index(query,'infobox',num_results)
    
 
    
    phrase_dict=defaultdict(float)
   
    
    for item in titles:
        phrase_dict[item[0]]+=(item[1])
        
    for item in bodys:
        phrase_dict[item[0]]+=(item[1])
    
    for item in refs:
        phrase_dict[item[0]]+=(item[1])
    
    for item in cats:
        phrase_dict[item[0]]+=(item[1])
    
    for item in infos:
        phrase_dict[item[0]]+=(item[1])
    
    
    
    top_ids=sorted(phrase_dict.items(),key=operator.itemgetter(1),reverse=True)
    #print(top_ids)
    top_results=[(ids[0],title_dict[ids[0]]) for ids in top_ids]

    return top_results

def search_field_query(query,num_results):
    query_list=re.split('(b:|i:|c:|r:|t:|l:)',query)
    #print(query_list)
    reference_q=None
    text_q=None
    category_q=None
    infobox_q=None
    title_q=None
    link_q=None

    reference_query=None
    text_query=None
    category_query=None
    infobox_query=None
    title_query=None
    link_query=None
    try:
        reference_q=query_list.index('r:')
    except:
        pass
    
    try:
        text_q=query_list.index('b:')
    except:
        pass
    
    try:
        category_q=query_list.index('c:')
    except:
        pass
        
    
    try:
        infobox_q=query_list.index('i:')
    except:
        pass
    
    
    try:
        title_q=query_list.index('t:')
    except:
        pass
    
    try:
        link_q=query_list.index('l:')
    except:
        pass
    
    rf=0
    bf=0
    tf=0
    cf=0
    if_=0
    lf=0
    

    if reference_q:
        rf=1
        reference_query=query_list[reference_q+1]
    
    if link_q:
        lf=1
        link_query=query_list[link_q+1]
    
    if text_q:
        bf=1
        text_query=query_list[text_q+1]

    if category_q:
        cf=1
        category_query=query_list[category_q+1]

    if infobox_q:
        if_=True
        infobox_query=query_list[infobox_q+1]

    if title_q:
        tf=1
        title_query=query_list[title_q+1]

    
    ref_res=[]
    info_res=[]
    title_res=[]
    cat_res=[]
    link_res=[]
    text_res=[]
    
    ref_title=[]
    info_title=[]
    title_title=[]
    cat_title=[]
    link_title=[]

    body_title=[]
    
    intersected_ids=[]

    field_query_start_time=time()
    
    if(reference_q):
        #print('r ',reference_query.rstrip())
        ref_res=search_in_index(reference_query,'references',num_results)
        ref_title=[(ids[0],title_dict[ids[0]]) for ids in ref_res]
        intersected_ids.append([ids[0] for ids in ref_res])

    if(link_q):
        #print('r ',reference_query.rstrip())
        link_res=search_in_index(link_query,'references',num_results)
        link_title=[(ids[0],title_dict[ids[0]]) for ids in link_res]
        intersected_ids.append([ids[0] for ids in link_res])
    
    if(infobox_q):
        #print('i ',infobox_query.rstrip())
        info_res=search_in_index(infobox_query,'infobox',num_results)
        info_title=[(ids[0],title_dict[ids[0]]) for ids in info_res]
        intersected_ids.append([ids[0] for ids in info_res])    
        
    if(text_q):
        #print('t ',text_query.rstrip())
        text_res=search_in_index(text_query,'body',num_results)
        body_title=[(ids[0],title_dict[ids[0]]) for ids in text_res]
        intersected_ids.append([ids[0] for ids in text_res])

    if(title_q):
        #print('T ',title_query.rstrip())
        title_res=search_in_index(title_query,'title',num_results)
        
        title_title=[(ids[0],title_dict[ids[0]]) for ids in title_res]
        intersected_ids.append([ids[0] for ids in title_res])
    
    if(category_q):
        #print('c ',category_query.rstrip())
        cat_res=search_in_index(category_query,'category',num_results)
        cat_title=[(ids[0],title_dict[ids[0]]) for ids in cat_res]
        intersected_ids.append([ids[0] for ids in cat_res])
        
    
    
    final_results=set(intersected_ids[0]).intersection(*intersected_ids)

    extras=[]

    results=[(item,title_dict[item]) for item in final_results]

    if len(results) >= num_results:
          field_query_end_time=time()

          total_time=field_query_end_time - field_query_start_time
          average_time_per_query=(total_time)/(if_+lf+tf+cf+bf+rf)
          
          return results,total_time,average_time_per_query

    else:
        if tf > 0:
            extras.append(search_phrase_query(title_query,num_results)[:num_results])

        if if_ > 0:
            extras.append(search_phrase_query(infobox_query,num_results)[:num_results])
        
        if cf > 0:
            extras.append(search_phrase_query(category_query,num_results)[:num_results])
        
        if lf > 0:
            extras.append(search_phrase_query(link_query,num_results)[:num_results])
        
        if rf > 0:
            extras.append(search_phrase_query(reference_query,num_results)[:num_results])
        
        if bf > 0:
            extras.append(search_phrase_query(text_query,num_results)[:num_results])
                

        union_results=list(set(extras[0]).union(*extras))
        
       

        results=results+union_results

        field_query_end_time=time()

        total_time=field_query_end_time - field_query_start_time
        average_time_per_query=(total_time)/(if_+lf+tf+cf+bf+rf)

    return results,total_time,average_time_per_query


def execute_query(query,num_results):
    if is_phrase_query(query):
        start=time()
        phrase_result=search_phrase_query(query,num_results)
        end=time()

        for i in range(num_results):
            result_file.write(f'{phrase_result[i][0]},{phrase_result[i][1]}\n')

        result_file.write(f'{end-start},{end-start}\n')
        result_file.write('\n')


        #print(f'Phrase Query Results \n {phrase_result[:num_results]}')
    
    else:
        #title_title,body_title,cat_title,info_title,ref_title,link_title=search_field_query(query,num_results)
        
        top_field_scores,total_time,average_time_per_query=search_field_query(query,num_results)
        


        for i in range(num_results):
            result_file.write(f'{top_field_scores[i][0]},{top_field_scores[i][1]}\n')

        result_file.write(f'{total_time},{average_time_per_query}\n')
        result_file.write('\n')
        # print(top_field_scores[:num_results])
        # print(total_time,average_time_per_query)



query_file=sys.argv[1]


query_file=open(query_file,'r',encoding='utf-8')
for queries in query_file:
    num_results,query=queries.split(',')
    print(num_results,query.rstrip())
    execute_query(query.rstrip(),int(num_results))
    print('')

query_file.close()


result_file.close()